### Test search functionality of Claude

In [3]:
import asyncio
from typing import TypedDict, List, Dict, Any, Union
from langgraph.graph import StateGraph, END
import anthropic
import json
from textwrap import dedent
from dataclasses import dataclass

# Initialize Anthropic client
client = anthropic.Anthropic()

# Define the state structure
class GraphState(TypedDict):
    user_query: str
    search_queries: List[str]
    search_results: List[Dict[str, Any]]
    final_summary: str

# Template for generating search queries
SEARCH_QUERY_TEMPLATE = """
    Given this user query: "{user_query}"

    Generate 3 diverse and complementary search queries that will help gather 
    comprehensive information to answer the user's question.

    Make the queries:
    - Specific and targeted
    - Cover different aspects of the topic
    - Likely to return different but complementary information

    Return ONLY a JSON array of 3 search query strings, like:
    ["query 1", "query 2", "query 3"]
"""

# Template for web search request
WEB_SEARCH_TEMPLATE = """
    Please search for information about: {search_query}

    Provide comprehensive information from your search results with proper citations.
"""

# Template for final summary
SUMMARY_TEMPLATE = """
    Based on the following search results for the user query: "{user_query}"

    Search Results:
    {combined_results}

    Please provide a comprehensive, well-structured answer that:
    1. Synthesizes information from all search results
    2. Highlights key findings and insights
    3. Notes any conflicting information
    4. Provides practical recommendations or conclusions
    5. Maintains proper citations where applicable

    Format the response with clear sections and make it easy to scan.
"""

def transform_to_search_queries(state: GraphState) -> GraphState:
    """Transform user query into 3 diverse search queries"""
    
    print(f"\n🎯 Transforming user query: {state['user_query']}")
    
    # Use Claude to generate 3 search queries
    prompt = dedent(SEARCH_QUERY_TEMPLATE).strip().format(user_query=state['user_query'])
    
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=512,
        messages=[{"role": "user", "content": prompt}]
    )
    
    try:
        # Extract the JSON array from Claude's response
        response_text = response.content[0].text
        search_queries = json.loads(response_text)
        
        # Ensure we have exactly 3 queries
        if len(search_queries) >= 3:
            state['search_queries'] = search_queries[:3]
        else:
            # Fallback if we don't get enough queries
            state['search_queries'] = search_queries + [state['user_query']] * (3 - len(search_queries))
    except Exception as e:
        print(f"⚠️ Error parsing queries: {e}")
        # Fallback to default queries if parsing fails
        state['search_queries'] = [
            state['user_query'],
            f"{state['user_query']} best practices",
            f"{state['user_query']} examples"
        ]
    
    print(f"\n🔍 Generated search queries:")
    for i, query in enumerate(state['search_queries'], 1):
        print(f"  {i}. {query}")
    
    return state

def parse_search_response(response) -> Dict[str, Any]:
    """Parse the complex response structure from Claude's web search"""
    
    result = {
        "full_response": "",
        "citations": [],
        "search_results": [],
        "tool_uses": [],
        "usage": {}
    }
    
    # Extract usage information
    if hasattr(response, 'usage') and response.usage:
        result["usage"] = response.usage.model_dump()
    
    # Process each content block
    for content_block in response.content:
        block_type = getattr(content_block, 'type', None)
        
        if block_type == 'text':
            # TextBlock
            result["full_response"] += content_block.text
            
            # Check for citations
            if hasattr(content_block, 'citations') and content_block.citations:
                for citation in content_block.citations:
                    result["citations"].append({
                        "cited_text": citation.cited_text,
                        "title": citation.title,
                        "url": citation.url
                    })
        
        elif block_type == 'server_tool_use':
            # ServerToolUseBlock
            result["tool_uses"].append({
                "id": content_block.id,
                "name": content_block.name,
                "input": content_block.input
            })
        
        elif block_type == 'web_search_tool_result':
            # WebSearchToolResultBlock
            if hasattr(content_block, 'content'):
                for search_result in content_block.content:
                    result["search_results"].append({
                        "title": search_result.title,
                        "url": search_result.url,
                        "page_age": search_result.page_age,
                        "type": search_result.type
                    })
    
    return result

def execute_searches(state: GraphState) -> GraphState:
    """Execute all search queries sequentially using Claude with web search tool"""
    
    print("\n🌐 Starting search execution...")
    
    search_results = []
    
    for idx, search_query in enumerate(state['search_queries'], 1):
        print(f"\n📍 Executing search {idx}/{len(state['search_queries'])}: {search_query}")
        
        try:
            prompt = dedent(WEB_SEARCH_TEMPLATE).strip().format(search_query=search_query)
            
            response = client.messages.create(
                model="claude-sonnet-4-20250514",
                max_tokens=2048,
                messages=[{"role": "user", "content": prompt}],
                tools=[{
                    "type": "web_search_20250305",
                    "name": "web_search",
                    "max_uses": 5  # Allow up to 5 searches per query
                }]
            )
            
            # Parse the complex response structure
            parsed_result = parse_search_response(response)
            
            result = {
                "query": search_query,
                "query_index": idx,
                "response": parsed_result["full_response"],
                "citations": parsed_result["citations"],
                "search_results": parsed_result["search_results"],
                "tool_uses": parsed_result["tool_uses"],
                "stop_reason": response.stop_reason,
                "usage": parsed_result["usage"]
            }
            
            search_results.append(result)
            print(f"✅ Completed search {idx}")
            
            # Print detailed information
            print(f"   📊 Tokens - Input: {parsed_result['usage'].get('input_tokens', 0)}, Output: {parsed_result['usage'].get('output_tokens', 0)}")
            print(f"   🔗 Found {len(parsed_result['search_results'])} web results")
            print(f"   📝 Generated {len(parsed_result['citations'])} citations")
            
        except Exception as e:
            print(f"❌ Error executing search {idx}: {str(e)}")
            search_results.append({
                "query": search_query,
                "query_index": idx,
                "response": f"Error executing search: {str(e)}",
                "error": True
            })
    
    state['search_results'] = search_results
    print(f"\n📊 All searches completed. Total results: {len(state['search_results'])}")
    
    return state

def summarize_results(state: GraphState) -> GraphState:
    """Summarize all search results into a comprehensive answer"""
    
    print("\n📝 Generating comprehensive summary...")
    
    # Prepare the search results for summarization
    all_results = []
    total_web_results = 0
    total_citations = 0
    
    for r in state['search_results']:
        if not r.get('error'):
            # Include the main response
            result_text = f"Search Query {r['query_index']}: {r['query']}\n"
            result_text += f"Results:\n{r['response']}\n"
            
            # Add web search metadata
            if 'search_results' in r:
                total_web_results += len(r['search_results'])
                result_text += f"\nWeb sources found ({len(r['search_results'])}):\n"
                for sr in r['search_results'][:3]:  # Show first 3 sources
                    result_text += f"  - {sr['title']} ({sr['url']})\n"
            
            # Add citation count
            if 'citations' in r:
                total_citations += len(r['citations'])
                result_text += f"\nCitations used: {len(r['citations'])}\n"
            
            result_text += f"{'-' * 40}"
            all_results.append(result_text)
    
    combined_results = "\n\n".join(all_results)
    
    # Use Claude to synthesize all results
    prompt = dedent(SUMMARY_TEMPLATE).strip().format(
        user_query=state['user_query'],
        combined_results=combined_results
    )
    
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=2048,
        messages=[{"role": "user", "content": prompt}]
    )
    
    state['final_summary'] = response.content[0].text
    
    # Calculate total usage
    total_input_tokens = sum(r.get('usage', {}).get('input_tokens', 0) for r in state['search_results'])
    total_output_tokens = sum(r.get('usage', {}).get('output_tokens', 0) for r in state['search_results'])
    
    print(f"\n📊 Total Usage Statistics:")
    print(f"   Input Tokens: {total_input_tokens}")
    print(f"   Output Tokens: {total_output_tokens}")
    print(f"   Web Results Found: {total_web_results}")
    print(f"   Citations Generated: {total_citations}")
    
    return state

def create_web_search_graph():
    """Create and compile the LangGraph workflow"""
    workflow = StateGraph(GraphState)
    
    # Add nodes
    workflow.add_node("transform_queries", transform_to_search_queries)
    workflow.add_node("execute_searches", execute_searches)
    workflow.add_node("summarize", summarize_results)
    
    # Define the flow
    workflow.set_entry_point("transform_queries")
    workflow.add_edge("transform_queries", "execute_searches")
    workflow.add_edge("execute_searches", "summarize")
    workflow.add_edge("summarize", END)
    
    # Compile the graph
    return workflow.compile()

# Helper function to run a search query
def run_web_search(query: str) -> Dict[str, Any]:
    """
    Run the complete web search pipeline for a given query.
    
    Args:
        query: The user's search query
        
    Returns:
        The final state containing search results and summary
    """
    # Create the graph
    app = create_web_search_graph()
    
    # Initialize state
    initial_state = {
        "user_query": query,
        "search_queries": [],
        "search_results": [],
        "final_summary": ""
    }
    
    # Run the graph
    print(f"🎯 User Query: {query}")
    print("=" * 60)
    
    try:
        final_state = app.invoke(initial_state)
        
        # Display the final summary
        print("\n" + "=" * 60)
        print("📋 FINAL COMPREHENSIVE ANSWER:")
        print("=" * 60)
        print(final_state['final_summary'])
        print("=" * 60)
        
        return final_state
        
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        print("\nTroubleshooting:")
        print("1. Check ANTHROPIC_API_KEY environment variable")
        print("2. Ensure web search is enabled in Anthropic Console")
        print("3. Verify you're using a supported model (claude-sonnet-4-20250514)")
        raise

# Function to display detailed results in a notebook-friendly format
def display_detailed_results(state: Dict[str, Any]):
    """Display detailed search results including web sources and citations"""
    print("\n🔍 Search Queries Used:")
    for i, query in enumerate(state['search_queries'], 1):
        print(f"  {i}. {query}")
    
    print(f"\n📊 Detailed Results:")
    for result in state['search_results']:
        if not result.get('error'):
            print(f"\n🔹 Query: {result['query']}")
            print(f"   - Web sources found: {len(result.get('search_results', []))}")
            print(f"   - Citations generated: {len(result.get('citations', []))}")
            
            # Show sample web sources
            if result.get('search_results'):
                print("   - Sample sources:")
                for sr in result['search_results'][:2]:
                    print(f"     • {sr['title']}")
                    print(f"       {sr['url']}")
    
    print("\n💡 Final Answer:")
    print("-" * 60)
    print(state['final_summary'])

# Example usage in Jupyter:
result = run_web_search("What are the top 5 IFRS 9 requirements in the context of credit risk modelling")
display_detailed_results(result)

🎯 User Query: What are the top 5 IFRS 9 requirements in the context of credit risk modelling

🎯 Transforming user query: What are the top 5 IFRS 9 requirements in the context of credit risk modelling

🔍 Generated search queries:
  1. IFRS 9 expected credit loss model requirements ECL calculation
  2. IFRS 9 three-stage approach credit risk assessment PD LGD EAD
  3. IFRS 9 forward-looking information macroeconomic scenarios credit risk modeling

🌐 Starting search execution...

📍 Executing search 1/3: IFRS 9 expected credit loss model requirements ECL calculation
✅ Completed search 1
   📊 Tokens - Input: 12646, Output: 1556
   🔗 Found 10 web results
   📝 Generated 22 citations

📍 Executing search 2/3: IFRS 9 three-stage approach credit risk assessment PD LGD EAD
✅ Completed search 2
   📊 Tokens - Input: 14567, Output: 1536
   🔗 Found 10 web results
   📝 Generated 24 citations

📍 Executing search 3/3: IFRS 9 forward-looking information macroeconomic scenarios credit risk modeling
✅ Compl

In [2]:
import anthropic

client = anthropic.Anthropic()

response = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "How do I update a web app to TypeScript 5.5?"
        }
    ],
    tools=[{
        "type": "web_search_20250305",
        "name": "web_search",
        "max_uses": 5
    }]
)
print(response)

Message(id='msg_01DFwj2kw772GNUvZfdkUHx8', content=[TextBlock(citations=None, text="I'll search for the latest information on updating to TypeScript 5.5 to give you the most current guidance.", type='text'), ServerToolUseBlock(id='srvtoolu_012mgibwtBxG2QsqeoScQ9z8', input={'query': 'TypeScript 5.5 update migration guide'}, name='web_search', type='server_tool_use'), WebSearchToolResultBlock(content=[WebSearchResultBlock(encrypted_content='EqsfCioIBBgCIiQ5ZDlkNzI1ZC05NTFkLTRmMTItYWMwMi0zOGUxZDAxZmNhZjkSDMHbtqvTF0sW2xsr0RoMOOPAwfmdTalaUeddIjBuMMwWwuLMLYu/IdfBhinpU7CGSVyGWBC+fhsNaBCpI6L1buVcUEPYmMSUdp0tTmUqrh6N4BOEIZfZwT6JnTLiR40AC94+4R5l8L+2GlWbGQYCrCBiohzk51op9jDELNS6kz3KoRZshoCXXnrQyFD6wBtvaUJZJPKrd+y5XDpU2XnJgVX5w2qvJtgwfaU72uKcAu9BTsux8tPeqKS7b0TMI4dFmMmERiFASaq4PyPQat3vzjnssnD0whjn5pzzoEmNt+4LtqqztHYkUldJQHDSU/sgP6tAgqh1FLfvgykeoI7ZhDHDuTPUVw4Vl4/gjg05Q6qDf7JHJBmBxD6BKCZ0hj2KItRI1mndYFxnABHdDRizom5pqCOMvKwuJfJIW8gCe9w2EPYvmy0ICc2TChoZbXe6cj8c96o5tt48lKWBVnZUOlNEtv54lZKXsV3l3KoWo63YE